In [ ]:
import pandas as pd
import chardet

def detect_file_encoding(file_path):
    with open(file_path, 'rb') as file:
        raw = file.read()
        result = chardet.detect(raw)
    return result['encoding'], result['confidence']

def read_csv_with_encoding(file_path, encoding):
    try:
        df = pd.read_csv(file_path, encoding=encoding)
        print(f"成功使用 {encoding} 讀取文件")
        print("前5行內容：")
        print(df.head().to_string())
        return df
    except UnicodeDecodeError:
        print(f"{encoding} 編碼無法正確解碼文件")
        return None

# 常用的中文編碼
chinese_encodings = ['big5', 'gb18030', 'gb2312', 'gbk', 'utf-8', 'cp950']

file_paths = ["V3_1131113.csv","B_V3_1131113.csv"]

for file_path in file_paths:
    print(f"\n處理文件：{file_path}")
    
    # 首先嘗試自動檢測
    detected_encoding, confidence = detect_file_encoding(file_path)
    print(f"自動檢測到的編碼：{detected_encoding}，置信度：{confidence}")
    
    # 嘗試自動檢測的編碼
    df = read_csv_with_encoding(file_path, detected_encoding)
    
    # 如果自動檢測失敗，嘗試其他中文編碼
    if df is None:
        for encoding in chinese_encodings:
            df = read_csv_with_encoding(file_path, encoding)
            if df is not None:
                break
    
    if df is None:
        print(f"無法找到正確的編碼來讀取 {file_path}")
    else:
        # 檢查是否有中文字符
        has_chinese = df.applymap(lambda x: '\u4e00' <= x <= '\u9fff' if isinstance(x, str) else False).any().any()
        if has_chinese:
            print("檢測到中文字符")
        else:
            print("警告：沒有檢測到中文字符，可能是編碼問題")

In [46]:
import pandas as pd

fileA=pd.read_csv("V3_1131113.csv",encoding = 'UTF-8-SIG')
fileB=pd.read_csv("B_V3_1131113.csv",encoding = 'Big5')


In [ ]:
import unicodedata

# 1. 先印出兩個檔案的欄位資訊
print("fileA 的欄位：")
print(fileA.columns.tolist())
print("\nfileB 的欄位：")
print(fileB.columns.tolist())

# 2. 檢查欄位名稱中的特殊字元
print("\n檢查欄位名稱：")
for col in fileB.columns:
    print(f"欄位名稱：'{col}'")
    print(f"字元編碼：{[ord(c) for c in col]}")

fileA_room_list = fileA.drop_duplicates(subset=["寢室"])["寢室"].reset_index(drop=True)

# 3. 嘗試修正欄位名稱
# 移除空格
fileB.columns = fileB.columns.str.strip()

# 轉換全形為半形
fileB.columns = [unicodedata.normalize('NFKC', col) for col in fileB.columns]

# 4. 再次檢查是否有「寢室」欄位
if '寢室' not in fileB.columns:
    print("\n警告：fileB(範本檔) 中仍然找不到「寢室」欄位")
    print("請檢查是否使用了不同的欄位名稱")
else:
    # 5. 如果修正成功，執行原本的檢查
    missing_rooms = set(fileA_room_list) - set(fileB["寢室"])
    if missing_rooms:
        print(f"\n警告：這些寢室在fileB(範本檔)中找不到：{missing_rooms}")
        print("目前的欄位有：", fileB.columns.tolist())

print(" ")
print(f"\nfileB(範本檔)資料預覽：")
print(fileB.head())

In [ ]:
fileA = fileA[fileA["金額"]>=0]
fileA_room_list = fileA.drop_duplicates(subset=["寢室"])["寢室"].reset_index(drop=True)
fileB_column = fileB.columns

for i in fileA_room_list:
    roommate_index = fileA[fileA["寢室"] == i].index
    new_room_index = fileB[fileB["寢室"] == i].index
    new_room_people = 0
    for j in roommate_index:
        for k in fileB_column:
            if(k in fileA.columns):
                fileB.loc[new_room_index[new_room_people],k] = fileA[k][j]
        new_room_people = new_room_people+1
        

In [ ]:
# 檢查寢室是否存在
missing_rooms = set(fileA_room_list) - set(fileB["寢室"])
if missing_rooms:
    print(f"Warning: These rooms are missing in fileB: {missing_rooms}")

# 檢查每個寢室的人數是否相符
for room in fileA_room_list:
    count_A = len(fileA[fileA["寢室"] == room])
    count_B = len(fileB[fileB["寢室"] == room])
    if count_A != count_B:
        print(f"Warning: Room {room} has different number of people: A={count_A}, B={count_B}")

In [ ]:
# 假設我們已經正確讀取了檔案
# fileB = pd.read_csv("B_V1_1130925.csv", encoding='big5')  # 或其他正確的編碼

# 輸出 CSV 檔案，指定 UTF-8 編碼
fileB.to_csv("名單程式轉換result_V3_1131113.csv", encoding='utf-8-sig', index=False)

# 驗證輸出
df_check = pd.read_csv("名單程式轉換result_V3_1131113.csv", encoding='utf-8-sig')
print(df_check.head())

# 檢查是否包含中文字符
has_chinese = df_check.applymap(lambda x: '\u4e00' <= x <= '\u9fff' if isinstance(x, str) else False).any().any()
if has_chinese:
    print("輸出的 CSV 檔案成功包含中文字符")
else:
    print("警告：輸出的 CSV 檔案可能沒有包含中文字符，請檢查原始數據")


In [8]:
fileB.to_csv("名單程式轉換result_V1_1131113.csv")